# Law, Bias, and Algorithms
## Algorithmic fairness (1/2)

In 2016, ProPublica published a [now-famous article](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) analyzing and criticizing the lack of fairness in a risk assessment tool used nation-wide called COMPAS.
Here, we will take a look at a cleaned-up version of the COMPAS data that ProPublica used, and try to better understand algorithmic fairness by investigating the claims ProPublica made, along with the [counter claims](https://www.propublica.org/article/technical-response-to-northpointe) made by NorthPointe (now re-branded as [Equivant](https://www.equivant.com/))

In [1]:
# Some initial setup
options(digits = 3)
library(tidyverse)

theme_set(theme_bw())

# Read the data
compas_df <- read_rds("../data/compas.rds")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## COMPAS data

A cleaned version of the COMPAS data is loaded as `compas_df`, with the following columns

* `id`: unique identifiers for each case
* `sex`, `dob`, `age`, `race`: demographic information for each defendant
* `decile_score`: COMPAS (decile) scores assessing risk that a defendant will recidivate within two years of release (higher score correspond to higher risk)
* `score_category`: COMPAS scores categorized into `Low`, `Medium`, and `High` risk
* `two_year_recid`: Indicator variable that is `1` if the defendant was arrested for a new crime within two years of release, and `0` otherwise.

In [36]:
compas_df

id,sex,dob,age,race,recid_score,violence_score,is_recid,is_violent_recid
<dbl>,<chr>,<date>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
3,Male,1982-01-22,34,African-American,3,1,1,1
4,Male,1991-05-14,24,African-American,4,3,1,0
5,Male,1993-01-21,23,African-American,8,6,0,0
8,Male,1974-07-23,41,Caucasian,6,2,1,0
10,Female,1976-06-03,39,Caucasian,1,1,0,0
13,Male,1994-06-10,21,Caucasian,3,5,1,1
14,Male,1988-06-01,27,Caucasian,4,4,0,0
15,Male,1992-08-18,23,African-American,6,4,1,0
16,Female,1978-08-22,37,Caucasian,1,1,0,0


In [27]:
compas_df %>%
    filter(is_recid == 1) %>%
    group_by(race) %>%
    summarize(mean(violence_score > 6))

compas_df %>%
    filter(violence_score > 6) %>%
    group_by(race) %>%
    summarize(mean(!is_recid))


race,mean(violence_score > 6)
<chr>,<dbl>
African-American,0.1355
Caucasian,0.0476


race,mean(!is_recid)
<chr>,<dbl>
African-American,0.263
Caucasian,0.321


### Popular definition of algorithmic fairness

There are three popular mathematical definitions of _fairness_:

1. __Calibration__: Outcome is independent of group membership given risk
2. __Classification Parity__: Model performance metrics (e.g., false positive rates) are equal across groups
3. __Anti-classification__: Protected characteristics (e.g., race) are not used by the algorithm.

While NorthPointe claims their algorithm does not use race information (anti-classification) as part of their model and their model is calibrated across racial groups (calibration), ProPublica points out that their model actually differs in false positive rates for different racial groups (classification parity).
In this notebook, we will examine some of their claims and aspects of fairness definitions and see why sometimes they are problematic formalizations of fairness definition.

## PLAN!
setup: we are going to explore... risk assessment
if performance boost between age/no age... -> use age?

WEEK 7

- ex1:
    - fit a model for compas data prior (+ age?) only. 
    - check acc

- ex2:
    - calibration by race: no difference
    - calculate manually
    - show them the plot
    
- ex3:
    - calculate/plot people detained vs. crime prevented
    - rank people using model, detain people with ~20+% risk (so that half the people are detained)
    - play with threshold
    - cost/benefit analysis
    - give the plot
    
- ex4:
    - raw disparity -- detaintion rate for race
    - assume 20% recidivism probability threshold
    - calculate the mean risk score and the mean recidism rate
    - plot risk distribution for them

- ex5: 
    - say we want 50% detaintion rate for both white & black, figure out what's the approrpirate thersholds

    
WEEK 8
recap the model first (just prior)

- ex1: 
    - calibration by then gender
    - first without plot -- student code

- ex2:
    - fit model prior + gender
    - check calibration again and check coef on gender
    
- ex3
    - FPR by race
    - FNR by race
    
- ex4
    - fix white threshold, equalize FPR by changing black threshold    

### Exercise 1 : False positive rates of the COMPAS score

In [37]:
# Calculate false positive rates for white and black 
# Show 31% vs. 15% FPR from slide 97 from the EC18/ICML18 fairness

### Exercise 2: Risk Distribution

In [40]:
# CODE: risk estimation code
# Plot estimated risk distribution (using gender + race + COMPAS score to predict recidivism) to show inframaginality problem
# borrow slides 67 and 68 figures

### Exercise 3: Check COMPAS model calibration

In [14]:
# CODE: Recalibration code
# slides 97 from the EC18/ICML18 fairness
# Make Calibration Plot for compas score
# for race
# for gender - it requires gender information to recalibrate
# train a logistic regression to re-calibrate the gender score

### Exercise 4: Recalibrate the model

In [41]:
# train a logistic regression to re-calibrate the gender score